## Working With Sessions


In [ ]:
# run this cell to avoid annoying warnings
import warnings
warnings.filterwarnings("ignore", message=r'.*numpy.dtype size changed*')

Import the LArray library:


In [ ]:
from larray import *

Check the version of LArray:

In [ ]:
from larray import __version__
__version__

### Before To Continue

If you not yet comfortable with creating, saving and loading sessions, please read first the [Creating Sessions](tutorial_presenting_larray_objects.ipynb#Creating-Sessions) and [Loading and Dumping Sessions](tutorial_IO.ipynb#Loading-and-Dumping-Sessions) sections of the tutorial before going further.  

### Exploring Content

To get the list of items names of a session, use the [names](../_generated/larray.Session.names.rst#larray.Session.names) shortcut (be careful that the list is sorted alphabetically and does not follow the internal order!):

In [ ]:
# load a session representing the results of a demographic model
filepath_hdf = get_example_filepath('population_session.h5')
s_pop = Session(filepath_hdf)

# print the content of the session
print(s_pop.names)

To get more information of items of a session, the [summary](../_generated/larray.Session.summary.rst#larray.Session.summary)  will provide not only the names of items but also the list of labels in the case of axes or groups and the list of axes, the shape and the dtype in the case of arrays:

In [ ]:
# print the content of the session
print(s_pop.summary())

### Selecting And Filtering Items

To select an item, simply use the syntax ``<session_var>.<item_name>``:

In [ ]:
s_pop.pop

To return a new session with selected items, use the syntax ``<session_var>[list, of, item, names]``:

In [ ]:
s_pop_new = s_pop['pop', 'births', 'deaths']

s_pop_new.names

The [filter](../_generated/larray.Session.filter.rst#larray.Session.filter) method allows you to select all items of the same kind (i.e. all axes, or groups or arrays) or all items with names satisfying a given pattern:

In [ ]:
# select only arrays of a session
s_pop.filter(kind=LArray)

In [ ]:
# selection all items with a name starting with a letter between a and k
s_pop.filter(pattern='[a-k]*')

### Arithmetic Operations On Sessions

Session objects accept binary operations with a scalar:

In [ ]:
# get population, births and deaths in millions
s_pop_div = s_pop / 1e6

s_pop_div.pop

with an array (please read the documentation of the [random.choice](../_generated/larray.random.choice.rst#larray.random.choice) function first if you don't know it):

In [ ]:
from larray import random
random_multiplicator = random.choice([0.98, 1.0, 1.02], p=[0.15, 0.7, 0.15], axes=s_pop.pop.axes)
random_multiplicator

In [ ]:
# multiply all variables of a session by a common array
s_pop_rand = s_pop * random_multiplicator

s_pop_rand.pop

with another session:

In [ ]:
# compute the difference between each array of the two sessions
s_diff = s_pop - s_pop_rand

s_diff.births

### Applying Functions On All Arrays

In addition to the classical arithmetic operations, the [apply](../_generated/larray.Session.apply.rst#larray.Session.apply) method can be used to apply the same function on all arrays. This function should take a single element argument and return a single value:

In [ ]:
# force conversion to type int
def as_type_int(array):
    return array.astype(int)

s_pop_rand_int = s_pop_rand.apply(as_type_int)

print('pop array before calling apply:')
print(s_pop_rand.pop)
print()
print('pop array after calling apply:')
print(s_pop_rand_int.pop)

It is possible to pass a function with additional arguments:

In [ ]:
# passing the LArray.astype method directly with argument 
# dtype defined as int
s_pop_rand_int = s_pop_rand.apply(LArray.astype, dtype=int)

print('pop array before calling apply:')
print(s_pop_rand.pop)
print()
print('pop array after calling apply:')
print(s_pop_rand_int.pop)

It is also possible to apply a function on non-LArray objects of a session. Please refer the documentation of the [apply](../_generated/larray.Session.apply.rst#larray.Session.apply) method.

### Comparing Sessions

Being able to compare two sessions may be useful when you want to compare two different models expected to give the same results or when you have updated your model and want to see what are the consequences of the recent changes.

[Session objects](../api.rst#session) provide the two methods to compare two sessions: [equals](../_generated/larray.Session.equals.rst#larray.Session.equals) and [element_equals](../_generated/larray.Session.element_equals.rst#larray.Session.element_equals).

The ``equals`` method will return True if **all items** from both sessions are identical, False otherwise:

In [ ]:
# load a session representing the results of a demographic model
filepath_hdf = get_example_filepath('population_session.h5')
s_pop = Session(filepath_hdf)

# create a copy of the original session
s_pop_copy = Session(filepath_hdf)

# 'equals' returns True if all items of the two sessions have exactly the same items
s_pop.equals(s_pop_copy)

In [ ]:
# create a copy of the original session but with the array
# 'births' slightly modified for some labels combination
s_pop_alternative = Session(filepath_hdf)
s_pop_alternative.births *= random_multiplicator

# 'equals' returns False if at least on item of the two sessions are different in values or axes
s_pop.equals(s_pop_alternative)

In [ ]:
# add an array to the session
s_pop_new_output = Session(filepath_hdf)
s_pop_new_output.gender_ratio = s_pop_new_output.pop.ratio('gender')

# 'equals' returns False if at least on item is not present in the two sessions
s_pop.equals(s_pop_new_output)

The ``element_equals`` method will compare items of two sessions one by one and return an array of boolean values:

In [ ]:
# 'element_equals' compare arrays one by one
s_pop.element_equals(s_pop_copy)

In [ ]:
# array 'births' is different between the two sessions
s_pop.element_equals(s_pop_alternative)

The ``==`` operator return a new session with boolean arrays with elements compared element-wise: 


In [ ]:
s_same_values = s_pop == s_pop_alternative

s_same_values.births

This also works for axes and groups:

In [ ]:
s_same_values.country

The ``!=`` operator does the opposite of ``==`` operator: 

In [ ]:
s_different_values = s_pop != s_pop_alternative

s_different_values.births 

A more visual way is to use the [compare](../_generated/larray.compare.rst#larray.compare) function which will open the ``Editor``.

```python
compare(s_pop, s_pop_alternative, names=['baseline', 'lower_birth_rate'])
```

![compare two sessions](../_static/compare_tutorial.png)

### Session API

Please go to the [Session](../api.rst#session) section of the API Reference to get the list of all methods of Session objects.